In [142]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [143]:
import os
import pandas as pd
import numpy as np

from math import isnan
from pathlib import Path
from datetime import datetime
from scipy.stats import bootstrap
from carla.data.catalog import OnlineCatalog

In [144]:
dataset_name = 'mnist'
model_type = "ann"
model_type_dict = "ann_linear"
data = OnlineCatalog(data_name=dataset_name)
num_features = len(data.continuous)
#result_path = 'results_remote/results'
result_path = os.path.join("~", "carla", "results")
remove_acdc = False
remove_vaes = False
vaes = ['crud','revise', 'clue']
sess_results_file = False

rms = "dice_cchvae_cruds_proto_roar_wachter_gs_acdc_retrained"
rms_array = ["dice", "cchvae", "cruds", "proto", "roar", "wachter", "gs", "acdc"]
date = str(datetime.date(datetime.now()))
save_results_path = 'tables/exp_1_CI/{}/'.format(date)
Path(save_results_path).mkdir(parents=True, exist_ok=True)

if dataset_name == 'breast_cancer' or dataset_name == 'give_me_some_credit':
    metrics = ['trade_off', 'LOF',  'L1_distance', 'Constraint_Violation']
    metrics_non_model = metrics.copy()
else:
    metrics = ['trade_off', 'LOF',  'L1_distance', 'avg_time']
    metrics_non_model = metrics.copy()

if model_type == 'forest':
    rms = 'acdc_dice_gs_proto_roar_retrained'
    rms_array = ['acdc', 'dice', 'gs', 'proto', 'roar']
elif dataset_name == 'mnist':
    rms = 'dice_proto_roar_wachter_gs_acdc_retrained'
    rms_array = ['dice', 'proto', 'roar', 'wachter', 'gs', 'acdc']

#rms = 'roar_retrained'
#rms_array = ['roar']

## Read results

In [145]:
recourse_results = pd.read_csv(result_path + '/{}/{}/{}/results.csv'.format(dataset_name, model_type_dict, rms))

* Add NaN for number of non successful -- Don't need this since it is only calc for successful. Need it for calculating success_rate
* use scipy to do bootstrapping

Start by testing bootstrapping on things we already have:


In [146]:
result_table = recourse_results[recourse_results['ML_Model'] == model_type]
result_table['L1_distance'] = result_table['L1_distance']/num_features
result_table['trade_off'] = np.sqrt(result_table['LOF'] * result_table['LOF'] + result_table['L1_distance'] * result_table['L1_distance'])

models = []
for type in ['ann', 'linear', 'forest']:
    for i in range(20):
        models.append("success_model_{}_{}".format(type, i))
    result_table["Success_retrained_{}_models".format(type)] = result_table[models].mean(axis=1)
    metrics.append("Success_retrained_{}_models".format(type))

/Users/mariamovin/.pyenv/versions/carla/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/mariamovin/.pyenv/versions/carla/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/mariamovin/.pyenv/versions/carla/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a co

In [147]:
result = pd.DataFrame(dtype="string")
for rm in rms_array:
    result_rm = pd.DataFrame(dtype="string")
    result_rm['Recourse_Method'] = [rm]
    for metric in metrics:
        print("----------------{}_________{}--------------".format(rm, metric))
        data = result_table[result_table['Recourse_Method'] == rm][metric]
        if data.min() == data.max():
            low = data.min()
            high = data.max()
            print("{},{}".format(data.mean(), data.mean()))
        else:
            data = (data, )
            res = bootstrap(data, np.mean)
            low = res.confidence_interval.low
            high = res.confidence_interval.high
            data = data[0]
            print(res.confidence_interval)
        result_rm[metric] = ['{:.2f} ({:.2f}, {:.2f})'.format(data.mean(), low, high )]

    result = pd.concat([result, result_rm])

----------------dice_________trade_off--------------
ConfidenceInterval(low=0.5069329860304171, high=0.6950892880949846)
----------------dice_________LOF--------------
ConfidenceInterval(low=0.49, high=0.69)
----------------dice_________L1_distance--------------
ConfidenceInterval(low=0.029610896222875466, high=0.043267256566213774)
----------------dice_________avg_time--------------
13.123525421620581,13.123525421620581
----------------dice_________Success_retrained_ann_models--------------
ConfidenceInterval(low=0.45600000000000007, high=0.5389999999999999)
----------------dice_________Success_retrained_linear_models--------------
ConfidenceInterval(low=0.33124999999999993, high=0.4195)
----------------dice_________Success_retrained_forest_models--------------
ConfidenceInterval(low=0.2733333333333333, high=0.3448333333333333)
----------------proto_________trade_off--------------
ConfidenceInterval(low=0.0430775015883189, high=0.04753873258023587)
----------------proto_________LOF---

In [148]:
success_result = pd.DataFrame()
for rm in rms_array:
    successful =int(result_table[result_table['Recourse_Method'] == rm]['Success_Rate'].mean() * 100)
    result_rm = pd.DataFrame(dtype="string")
    result_rm['Recourse_Method'] = [rm]
    if successful == 100:
        result_rm['Success_Rate'] = ['{:.2f} ({:.2f}, {:.2f})'.format(1.0, 1.0, 1.0 )]
    else:
        success_table = np.array([1]*successful + [0] * (100-successful))
        data = (success_table, )
        res = bootstrap(data, np.mean)
        low = res.confidence_interval.low
        high = res.confidence_interval.high
        data = data[0]
        print(res.confidence_interval)
        result_rm['Success_Rate'] = ['{:.2f} ({:.2f}, {:.2f})'.format(data.mean(), low, high )]
    success_result = pd.concat([success_result, result_rm])

result = result.set_index('Recourse_Method')
success_result = success_result.set_index('Recourse_Method')
result = result.join(success_result, on='Recourse_Method')
#success_table = pd.DataFrame([1]*successful + [0] * (100-successful))
#result_table[['Success_Rate', "Recourse_Method"]].groupby('Recourse_Method').mean()

ConfidenceInterval(low=0.26, high=0.45)


In [149]:
#result['Success_Rate'] = result_table[['Success_Rate', "Recourse_Method"]].groupby('Recourse_Method').mean()['Success_Rate']

In [150]:
#order_of_columns = ['Success_Rate', "Success_retrained_ann_models", "Success_retrained_linear_models", "Success_retrained_forest_models", "trade-off", "LOF", "L1_Distance"]
order_of_columns = ['Success_Rate', "Success_retrained_ann_models", "Success_retrained_linear_models", "Success_retrained_forest_models"] + metrics_non_model
#result = result.join(result_table[['Success_Rate', "Recourse_Method"]].groupby('Recourse_Method').mean(), on='Recourse_Method').set_index('Recourse_Method')
result = result.sort_index()
result = result[order_of_columns]
result.to_csv('{}{}_{}.csv'.format(save_results_path, dataset_name, model_type + '_time'), float_format='%.2f')

#small_result_table.to_csv('{}{}_{}_small.csv'.format(save_results_path, dataset_name, model_type), float_format='%.2f')